In [96]:
import pandas as pd

In [97]:
togo_cleaned = pd.read_parquet(
    '/data/eop/Togo 2018-19/clean/final_togo.parquet'
)
togo_summary = pd.read_parquet(
    '/data/eop/Togo 2018-19/clean/summary.parquet'
)
cdr_matching = pd.read_csv(
    '/data/togo_anon/surveys/survey2018outcomes.csv'
)[['hhid', 'phone_number']]

In [98]:
merged = togo_cleaned.merge(
    cdr_matching,
    how='inner',
    on='hhid',
)

In [ ]:
feats = pd.read_csv(
    '/data/togo_anon/feats/feats_2018.csv'
)


In [103]:
togo_with_feats.has_phone_data.value_counts()

has_phone_data
True     4262
False    1893
Name: count, dtype: int64

In [ ]:
1+1893

6155

In [99]:
togo_with_feats = merged.merge(
    feats,
    left_on='phone_number',
    right_on='name',
    how='left',
    indicator=True,
).drop(columns=['name'])
togo_with_feats['_merge'] = togo_with_feats['_merge'] == 'both'
togo_with_feats.rename(columns={'_merge': 'has_phone_data'}, inplace=True)

In [100]:
togo_with_feats.drop(columns=[c for c in feats.columns if c.startswith('reporting')], inplace=True)
new_columns = []

for c in feats.columns:
    if c == 'name': 
        continue
    assert togo_with_feats[c].dtype in ['int64', 'float64'], f"Column {c} is not numeric"
    new_columns.append(togo_with_feats[c].isna().rename(f"{c}_missing"))

    togo_with_feats.fillna({c: togo_with_feats[c].mean()}, inplace=True)

togo_with_feats = pd.concat([togo_with_feats] + new_columns, axis=1)

In [101]:
cdr_feature_columns = list(feats.columns.difference(
    ['name']
))
universal_columns = [
    'hhid', 'hh_wgt', 'headcount_adjusted_hh_wgt', 'consumption_per_capita_per_day'
]
summary_of_phone_columns = pd.DataFrame(
    {
        'variable_name': cdr_feature_columns,
        'data_type': len(cdr_feature_columns) * ['numeric'],
        'geographic_indicator': len(cdr_feature_columns) * [False],
        'geographic_indicator_finer': len(cdr_feature_columns) * [False],
        'geographic_indicator_coarser': len(cdr_feature_columns) * [False],
    }
)

summary_of_universal_columns = togo_summary[togo_summary.variable_name.isin(universal_columns)]

In [102]:

togo_with_feats[universal_columns + cdr_feature_columns].to_parquet(
    '/data/eop/Togo 2018-19/clean/cdr_features/togo.parquet', index=False
)
pd.concat(
    [summary_of_phone_columns, summary_of_universal_columns]
).drop_duplicates(subset='variable_name').to_parquet(
    '/data/eop/Togo 2018-19/clean/cdr_features/summary.parquet', index=False
)

togo_with_feats[list(togo_cleaned.columns) + cdr_feature_columns].to_parquet(
    '/data/eop/Togo 2018-19/clean/cdr_features_and_survey_predictors/togo.parquet', index=False
)
pd.concat(
    [togo_summary, summary_of_phone_columns]
).drop_duplicates(subset='variable_name').to_parquet(
    '/data/eop/Togo 2018-19/clean/cdr_features_and_survey_predictors/summary.parquet', index=False
)